# Repetition from Day 1
ToDo: Which topics to repeat? Prepare here?

# Day 2, before lunch: Time series analysis


Next step: Read stuff using pandas, making use of it's time series analysis and statistics functions

In [ ]:
import pandas as pd

df = pd.read_table(
    'd1s2/rr24_Bulken.txt', encoding='utf8', 
    header=13,
    skipfooter=12, engine='python',  # The file contains 12 footer lines, and skipfooter requires the python parsing engine
    sep='\s+',                       # Column separtion by one or more whitespace
    parse_dates=[1,], dayfirst=True, # Second column contains dates, in European format
    index_col=1,                     # Use the date column as index
)

# Note the use of column headers as attributes!
# df.RR.plot()
df

What is this ``df.RR``, and what else can we do with it?

In [ ]:
type(df.RR)

In [ ]:
dir(df.RR)

Many things available! We'll dive into a few of them later. 

First, a quick and easy first data analysis: Cumulative precipitation during that period.

In [ ]:
type(df.RR.cumsum())

Note: ``cumsum()`` returns a time series, so we can work with that in exactly the same way as ``df.RR``.

In [ ]:
df.RR.cumsum().plot()

Second example, find dates where the 24-hour precipitation exceeded 50 mm.

In [ ]:
df.index[df.RR > 50]

In [ ]:
df.loc[df.RR > 50]

In [ ]:
df.iloc[45]

Worth taking some time to figure out in detail what happens here.

Todo: Give some more details also here in text. Explain the above df.loc, df.iloc

In [ ]:
type(df.RR > 50), (df.RR > 50).dtype, len(df.RR)

We're using a boolean time series to select dates.

We could also use any other (random) boolean time series of length 100.

In [ ]:
import numpy as np

randombools = pd.Series(np.random.rand(100) >= 0.9, index=df.RR)
randombools.sum() # How many True values (=ones) are there in the time series?

In [ ]:
df.index[randombools]

## Comparing time series

So far we have only worked with one time series. Let's add a second to have some more analysis options to explore.

Unfortunately, the date format of ``rro_Bulken.txt`` is not recognised automatically by pandas, so we need to supply our custom conversion function.

ToDo: Explain the ``lambda``.

In [ ]:
from datetime import datetime

date_parser = lambda datestr: datetime.strptime(datestr, '%d%m%Y')

This ``lambda`` is essentially just a shorthand for defining a function. We can use ``date_parser`` just as any other function.

In [ ]:
date_parser('23012019')

In [ ]:
df2 = pd.read_table('d1s2/rro_Bulken.txt', encoding='latin1', 
                   header=None, names=['Dato', 'Level', 'Discharge', 'p75', 'p50', 'p25'],
                                                    # Custom header information
                   comment='#',                     # Ignore lines starting with #
                   na_values=['----', ],            # Custom marker for missing values
                   sep='\s+',                       # Column separtion by one or more whitespace
                   parse_dates=[0,], date_parser=date_parser, 
                                                    # First column contains dates, custom format
                   index_col=0,                     # Use the date column as index
)

A quick sanity check to see whether we got what we expected.

In [ ]:
df2.Discharge.plot()

### Correlation analyses

Is river runoff correlated to precipitation?

In [ ]:
# Note the different time periods for df and df2!
df.RR.corr(df2.Discharge)

Seems like!

But what about the median discharge and precipitation?

In [ ]:
df2.p50.corr(df.RR)

More advance statistics will require the ``scipy.stats`` module. But to be able to use that module, we'll need to homogenise the two time series:
 * Identical index, i.e. dates
 * Remove NaNs
 
Once complete, we'll then make use of ``scipy.stats`` to estimate the significance of the above correlations and calculate lagged correlations between the two time series.

In [ ]:
# First combine into common data frame, using the same index
dfa = pd.concat([df, df2], axis=1)

In [ ]:
dfa

In [ ]:
# Then extract the columns we are interested in, keeping only those rows where we have data in both
df_clean = dfa[['RR', 'Discharge']].dropna()

In [ ]:
df_clean

Now we're finally ready to calculate the Pearson correlaton including it's significance. Documentation for the function is given here:

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr

In [ ]:
import scipy.stats

scipy.stats.pearsonr(df_clean.RR, df_clean.Discharge)

Cool, seems highly significant!

In [ ]:
df_clean = dfa[['RR', 'p50']].dropna()
scipy.stats.pearsonr(df_clean.RR, df_clean.p50)

While the correlation between median discharge and precipitation seems spurious.

With that clarified, onto lagged correlations.

### Exercise 1: Lagged correlations

At which lead/lag in days does the correlation between precipitation reach it's maximum?

The ``pandas.Series.shift`` function might come in handy for the analysis.

Todo: Break up into several steps, smaller assignment pieces

In [ ]:
tshifts = np.arange(-5,6)
corrcoefs = np.empty(tshifts.shape)
pvalues = np.empty(tshifts.shape)
for i, tshift in zip(range(len(tshifts)), tshifts):
    RRshift = df.RR.shift(tshift)
    dfs = pd.concat([RRshift, df2.Discharge], axis=1)
    dfs = dfs.dropna()
    corrcoefs[i], pvalues[i] = scipy.stats.pearsonr(dfs.RR, dfs.Discharge)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(tshifts, corrcoefs, linewidth=1, color='C0')
plt.plot(tshifts[pvalues < 0.01], corrcoefs[pvalues < 0.01], linewidth=3, color='C0')
plt.xlabel('<- Discharge leads / Rain leads -> [days]')
plt.ylabel('Correlation coefficient')

### Exercise 2: Smoothing time series

The river discharge might contain an integral of the precipitation over the preceeding days. Let's correlate smoothed precipitation with discharge, to see whether we can further increase the correlations. Use running means of 1-5 days centred on the given date in combination with lags between -5 and 5 days to find the maximum correlation.

The ``pandas.Series.rolling`` function might come in handy for calculating the running mean.

In [ ]:
rmeans = np.arange(1,6)
tshifts = np.arange(-5,6)
corrcoefs = np.empty(rmeans.shape+tshifts.shape)
pvalues = np.empty(rmeans.shape+tshifts.shape)
for j, rmean in zip(range(len(rmeans)), rmeans):
    RRmean = df.RR.rolling(window=rmean, center=True).mean()
    for i, tshift in zip(range(len(tshifts)), tshifts):
        RRshift = RRmean.shift(tshift)
        dfs = pd.concat([RRshift, df2.Discharge], axis=1)
        dfs = dfs.dropna()
        corrcoefs[j,i], pvalues[j,i] = scipy.stats.pearsonr(dfs.RR, dfs.Discharge)

In [ ]:
for j, rmean in zip(range(len(rmeans)), rmeans):
    plt.plot(tshifts, corrcoefs[j], linewidth=1, color='C%d' % j)
    plt.plot(tshifts[pvalues[j,:] < 0.01], corrcoefs[j,pvalues[j,:] < 0.01], 
             linewidth=3, color='C%d' % j, label='%d-day mean' % rmean)
plt.xlabel('<- Discharge leads / Rain leads -> [days]')
plt.ylabel('Correlation coefficient')
plt.legend()

### Exercise 3: Fitting linear model

For the combination of running mean and time lag that yields the maximum correlation create a linear model to estimate discharge from observed precipitation. Visualise the results in a scatter as well as a time-series plot comparing predicted and actual discharge.

ToDo: Break up into pieces

In [ ]:
# Explain the following, make it NOT part of the assignment.
jmax, imax = np.unravel_index(np.argmax*(corrcoefs), corrcoefs.shape)
print('Maximum correlation of %3.1f%% at rmean %d days and lag %d days' % (corrcoefs.max()*100, rmeans[jmax], tshifts[imax]))

Recreate the shifted running mean that gave the maximum correlation 

In [ ]:
RRmax = df.RR.rolling(window=rmeans[jmax], center=True).mean().shift(tshifts[imax])

Collect all relevant ``Series`` in a new data frame

In [ ]:
# Rename column to avoid having two columns named "RR", then concatenate to one dataframe
RRmax.rename('RRmax', inplace=True)
dfs = pd.concat([RRmax, df.RR, df2.Discharge], axis=1)
dfs_clean = dfs.dropna()

Fit the linear model by linear regression

Todo: Here the actual assignment starts, give hints ``scipy.stats.linregress``. May be make a standard function from the labmda.

In [ ]:
linmodel = scipy.stats.linregress(dfs_clean.RRmax, dfs_clean.Discharge)
estimate_Discharge = lambda RRm: linmodel.slope*RRm + linmodel.intercept
dfs['Discharge_linmodel'] = pd.Series(estimate_Discharge(dfs.RRmax), index=dfs.index)

Scatter plot for evaluation

Todo: Hint ``scatter``.

In [ ]:
plt.scatter(dfs.Discharge, dfs.Discharge_linmodel)
plt.plot([0, 500], [0, 500], 'k-', linewidth=1)
plt.xlabel('Observed discharge [m3/s]')
plt.ylabel('Estimated discharge from precipitation [m3/s]')

Time series plot for evaluation

Hint: use ``label = 'String'`` as a keyword argument to the plot. Doublecheck whether legend is new.

In [ ]:
dfs_clean = dfs.dropna()
dfs_clean.Discharge.plot(label='Observed')
dfs_clean.Discharge_linmodel.plot(label='Estimated')
plt.ylabel('Discharge [m3/s]')
plt.legend()